[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/02-dynamic-routes.ipynb)

# Dynamic Routes via OpenAI Function Calling

In this Notebook we showcase an alternative to dynamic routes where an `openai_function_schema` is specified or the Route, and in doing so, OpenAIs "Function Calling" API is utilized. 

The usual Semantic Router procedure is used to determine which Route is most appropriate for the input utterance. Then, rather than using a regular LLM call with careful prompt engineering to allow the LLM to extract function arguments from the input, OpenAIs Function Calling is leveraged to try and obtain function arguments more reliably. 


## Installing the Library

In [2]:
!pip install -qU semantic-router==0.0.34


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Initializing Routes and RouteLayer

Dynamic routes are treated in the same way as static routes, let's begin by initializing a `RouteLayer` consisting of static routes.

In [3]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We initialize our `RouteLayer` with our `encoder` and `routes`. We can use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`, or local alternatives like `FastEmbedEncoder`.

In [4]:
import os
from getpass import getpass
from semantic_router import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder

# dashboard.cohere.ai
# os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
#     "Enter Cohere API Key: "
# )
# platform.openai.com
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

# encoder = CohereEncoder()
encoder = OpenAIEncoder()

rl = RouteLayer(encoder=encoder, routes=routes)

2024-04-29 01:48:49 INFO semantic_router.utils.logger local


We run the solely static routes layer:

In [5]:
rl("how's the weather today?")

RouteChoice(name='chitchat', function_call=None, similarity_score=None)

## Creating a Dynamic Route

As with static routes, we must create a dynamic route before adding it to our route layer. To make a route dynamic, we need to provide a `function_schema`. The function schema provides instructions on what a function is, so that an LLM can decide how to use it correctly.

In [6]:
from datetime import datetime
from zoneinfo import ZoneInfo


def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")

In [7]:
get_time("America/New_York")

'17:48'

To get the function schema we can use the `get_schema` function from the `function_call` module.

In [8]:
from semantic_router.utils.function_call import get_schema

schema = get_schema(get_time)
schema

{'name': 'get_time',
 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
 'signature': '(timezone: str) -> str',
 'output': "<class 'str'>"}

In [9]:
from semantic_router.utils.function_call import get_schema_openai_func_calling

openai_function_schema = get_schema_openai_func_calling(get_time)
openai_function_schema

{'type': 'function',
 'function': {'name': 'get_time',
  'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
  'parameters': {'type': 'object',
   'properties': {'timezone': {'type': 'string',
     'description': 'The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.'}},
   'required': ['timezone']}}}

We use this to define our dynamic route:

In [10]:
time_route = Route(
    name="get_time",
    utterances=[
        "what is the time in new york city?",
        "what is the time in london?",
        "I live in Rome, what time is it?",
    ],
    # function_schema=schema,
    openai_function_schema=openai_function_schema,
)

Add the new route to our `layer`:

In [11]:
rl.add(time_route)

2024-04-29 01:48:50 INFO semantic_router.utils.logger Adding `get_time` route


Now we can ask our layer a time related question to trigger our new dynamic route.

In [12]:
out = rl("what is the time in new york city?")
out

2024-04-29 01:48:52 WARNING semantic_router.utils.logger No LLM provided for dynamic route, will use OpenAI LLM default. Ensure API key is set in OPENAI_API_KEY environment variable.


RouteChoice(name='get_time', function_call={'timezone': 'America/New_York'}, similarity_score=None)

In [13]:
get_time(**out.function_call)

'17:48'

Our dynamic route provides both the route itself _and_ the input parameters required to use the route.

Now let's just verify that the other route selection still work correctly, starting with chitchat.

In [14]:
out = rl("What's the weather like?")
out

RouteChoice(name='chitchat', function_call=None, similarity_score=None)

Now, an utterance that shouldn't fall into any of the routes defined above.

In [15]:
out = rl("Tell me how to bake cake?")
out

RouteChoice(name=None, function_call=None, similarity_score=None)

---